In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import torch.optim.optimizer as optimizer
import src.load as load
import src.training as training
import optuna
import random

from src.model import DistModel, ScheduleModel, KiloModel, ClimateEncoder
from src.dataset import HarvestDataset
import plotly.graph_objects as go

train_dataset, test_dataset, mappings, reverse_mappings, train_meta, test_meta  = load.separate_prop('../data/processed/')
train_dataset_year, test_dataset_year, mappings_year, reverse_mappings_year, train_meta_year, test_meta_year  = load.separate_year('../data/processed/')

In [3]:
hyperparameters = {
    'lr': [1e-4, 1e-3, 1e-2, 1e-5],
    'encoder_lr': [1e-4, 1e-3, 1e-2, 1e-5],
    'encoder_gru_lr': [1e-4, 1e-3, 1e-2, 1e-5],
    'encoder_loc_lr': [1e-4, 1e-3, 1e-2, 1e-5],
    'encoder_type_lr': [1e-4, 1e-3, 1e-2, 1e-5],
    'kilo_gru_lr': [1e-4, 1e-3, 1e-2, 1e-5],
    'kilo_out_lr': [1e-4, 1e-3, 1e-2, 1e-5],
    'weight_decay': [1e-4, 1e-3, 1e-2, 1e-5],
    'dropout': [0.1, 0.2, 0.3, 0.4],
    'step_size': [10, 20, 30, 40],
    'gamma': [0.1, 0.2, 0.3, 0.4],
    'batch_size': [32,64, 128, 256],
    'patience':[10,20,30],
    'hidden_size':[32,64,128,256],
    'num_layers':[2,3,4,5],
    'climate_hidden_dim':[32,64,128,256],
    'climate_num_layers':[2,3,4,5],
    'climate_dropout':[0.1,0.2,0.3,0.4],
    'ranch_dim':[4,8,12,16],
    'parcel_dim':[4,8,12,16],
    'class_dim':[4,8,12,16],
    'type_dim':[4,8,12,16],
    'variety_dim':[4,8,12,16]
}

In [4]:
def train_dist(model,num_weeks, optimizer, scheduler, config):
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)
    
    criterion = nn.MSELoss()
    model.train()

    for epoch in range(config['num_epochs']):
        for batch in train_loader:
            features, encoded_features, climate_data, kilo_gru_input, kilo_dist, log1p_kilos, _,_, _ = batch
            kilo_gru_input = kilo_gru_input[:,:num_weeks,:]

            dist_output = model(features, encoded_features, climate_data, kilo_gru_input)
            if torch.isnan(dist_output).any():
                raise ValueError('NaN in dist output')
            
            pred_input = torch.expm1(dist_output[:,num_weeks:].cumsum(dim=1) * log1p_kilos.unsqueeze(-1))
            actual_input = torch.expm1(kilo_dist[:,num_weeks:].cumsum(dim=1) * log1p_kilos.unsqueeze(-1))

            loss = criterion(pred_input, actual_input)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)

            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()
        scheduler.step()

    val_loss = 0
    model.eval()
    with torch.no_grad():
        for batch in val_loader:
            features, encoded_features, climate_data, kilo_gru_input, kilo_dist, log1p_kilos, log1p_schedule,_,_ = batch
            kilo_gru_input = kilo_gru_input[:,:num_weeks,:]
            kilo_output = model(features, encoded_features, climate_data, kilo_gru_input)
            pred_input = torch.expm1(kilo_output[:,num_weeks:].cumsum(dim=1) * log1p_kilos.unsqueeze(-1))
            actual_input = torch.expm1(kilo_dist[:,num_weeks:].cumsum(dim=1) * log1p_kilos.unsqueeze(-1))

            loss = criterion(pred_input, actual_input)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    return val_loss

In [5]:
NUM_WEEKS = 1

In [ ]:
def objective(trial):
    config = {

        'gamma': trial.suggest_float("gamma", 0.1, 1.0,log=True),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-2,log=True),
        "step_size": trial.suggest_int("step_size", 5, 30),
        "climate_dropout": trial.suggest_float("climate_dropout", 0.1, 0.5),
        "embedding_dim": trial.suggest_categorical("embedding_dim", [4, 8, 12, 16]),
        "num_epochs": 50,
        "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128]),
        "base_lr": trial.suggest_float("base_lr", 1e-5, 1e-2,log=True)
    }

    lr = config["base_lr"] * (config["batch_size"] / 32)
    for _ in range(5):
        try:
            model = DistModel(
                batch_size=config["batch_size"],
                dropout=config["dropout"],
                climate_dropout=config["climate_dropout"],
                embedding_dim=config["embedding_dim"]
            )
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=config["weight_decay"])
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config["step_size"], gamma=config["gamma"])
            val_loss = train_dist(model, NUM_WEEKS, optimizer, scheduler, config)
            break
        except ValueError as e:
            print(e)
    return round(val_loss, 4)

In [15]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

print("Best hyperparams:", study.best_params)

[I 2025-07-15 12:33:44,318] A new study created in memory with name: no-name-babfc90f-6eed-4464-975c-1702e7817cfa
[I 2025-07-15 12:34:03,570] Trial 0 finished with value: 2262418.2292 and parameters: {'gamma': 0.5374630205986435, 'dropout': 0.11638082098598393, 'weight_decay': 2.5289350512576507e-06, 'step_size': 25, 'climate_dropout': 0.17773761357671025, 'embedding_dim': 4, 'batch_size': 128, 'base_lr': 0.0001842616497976055}. Best is trial 0 with value: 2262418.2292.
[I 2025-07-15 12:34:44,534] Trial 1 finished with value: 2109582.6237 and parameters: {'gamma': 0.13790821317906865, 'dropout': 0.11847257508376385, 'weight_decay': 0.0020709328345362757, 'step_size': 30, 'climate_dropout': 0.27584230164656565, 'embedding_dim': 12, 'batch_size': 32, 'base_lr': 0.00028194501373948473}. Best is trial 1 with value: 2109582.6237.
[I 2025-07-15 12:35:10,541] Trial 2 finished with value: 2173780.724 and parameters: {'gamma': 0.22454237559184617, 'dropout': 0.27109094410303974, 'weight_decay':

Best hyperparams: {'gamma': 0.13790821317906865, 'dropout': 0.11847257508376385, 'weight_decay': 0.0020709328345362757, 'step_size': 30, 'climate_dropout': 0.27584230164656565, 'embedding_dim': 12, 'batch_size': 32, 'base_lr': 0.00028194501373948473}


In [16]:
optuna.visualization.plot_param_importances(study)



In [17]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)